In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
project = r"C:\Users\C62216A\Documents\Non Project\KS sessions"

In [ ]:
df = pd.read_csv(os.path.join(project, "museums.csv"))

In [ ]:
# check the first 5 rows
df.head()

In [ ]:
# check the number of obs and the number of columns
df.shape

In [ ]:
# list all the variable names in the dataset
list(df)

In [ ]:
# check the data types of your variable
df.dtypes

In [ ]:
# Check Full dups
df['full_dups']=df.duplicated()
df['full_dups'].value_counts()
# No full dups

In [ ]:
df = df[df['full_dups'] == False]
print(df.shape)

In [ ]:
df['full_dups'].value_counts()

In [ ]:
df['duplicate_museum_id']=df.duplicated('Museum ID')
df['duplicate_museum_id'].value_counts()
# No dups by ID

In [ ]:
#Drop the 2 columns created for Duplicate checks from the dataset
#Drop a variable and create a new dataset
df = df.drop(['duplicate_museum_id','full_dups'], axis=1).copy()
print(df.shape)

In [ ]:
# Example of regular funciton

def double(x):
    return x*2

df["museum_id_double_1"] = double(df["Museum ID"])
df[["Museum ID", 'museum_id_double_1']].head()

In [ ]:
# Example of Lambda function
# additional reading : https://www.programiz.com/python-programming/anonymous-function

df["museum_id_double_2"]=df["Museum ID"].apply(lambda x: x * 2)
df[["Museum ID", 'museum_id_double_1', "museum_id_double_2"]].head()

In [ ]:
df = df.drop(['museum_id_double_1','museum_id_double_2'], axis=1).copy()
print(df.shape)

In [ ]:
df.head()

In [ ]:
df.sort_values(by='Revenue',ascending=True)

In [ ]:
# check what is the maximum revenue per state

# Additional Reading : https://pbpython.com/pandas_transform.html

df['max_revenue_per_state'] = df.groupby(["State (Administrative Location)"])["Revenue"].transform('max')

In [ ]:
# cross-tab of the maximum revenue per state and the state

pd.crosstab(df['State (Administrative Location)'],df['max_revenue_per_state'])

In [ ]:
# check the first 10 rows of Revenue in Los Angeles
df[df['City (Administrative Location)'] == 'LOS ANGELES']['Revenue'].head(10)

In [ ]:
# we can group by City and State and fill in the Revenue with the one of the previous / next record

# backwards fill
df['revenue_filled'] = df.groupby(['City (Administrative Location)'])['Revenue'].bfill()

# check the first 10 rows of filled in Revenue in Los Angeles
df[df['City (Administrative Location)'] == 'LOS ANGELES']['revenue_filled'].head(10)

In [ ]:
# forward fill
df['revenue_filled'] = df.groupby(['City (Administrative Location)'])['revenue_filled'].ffill()

# check the first 10 rows of filled in Revenue in Los Angeles
df[df['City (Administrative Location)'] == 'LOS ANGELES']['revenue_filled'].head(10)

In [ ]:
# check quickly some statistics for a numeric variable

df['revenue_filled'].describe()

In [ ]:
df['Revenue'].describe()

In [ ]:
# check what is the mean revenue per City after we have filled in the missings
# Additional Reading : https://pbpython.com/pandas_transform.html

df['mean_revenue_per_city'] = df.groupby(["City (Administrative Location)"])["revenue_filled"].transform('mean')

In [ ]:
pd.options.display.precision = 45
df[df['City (Administrative Location)'] == 'LOS ANGELES']['mean_revenue_per_city'].value_counts(dropna=False)

In [ ]:
# Alternative way of getting the number of museums , mean and max revenue per city in a report

df.groupby(["City (Administrative Location)"]) \
    .agg(total = ('City (Administrative Location)', 'count'),
         mean_revenue = ('revenue_filled', 'mean'),
         max_revenue = ('revenue_filled', 'max')) \
    .reset_index() \
    .to_excel(project + '\\' + 'Report.xlsx')

In [ ]:
df['State (Administrative Location)'].value_counts(dropna=False)

In [ ]:
# create a subset with only records that match a specific condition

subset_NY_CA = df[df['State (Administrative Location)'].isin(['NY',
                                                              'CA'])]

In [ ]:
subset_NY_CA['State (Administrative Location)'].value_counts(dropna=False)

In [ ]:
subset_NY_CA.to_csv(r'C:\Users\C62216A\Documents\Non Project\KS sessions\subset.csv',  index = False)

In [ ]:
# Creating a flag variable

df['missing_revenue_flag'] = np.where(df['Revenue'].isna(),
                                     "Missing Revenue",
                                     "Not Missing Revenue")

In [ ]:
df[['missing_revenue_flag', 'Revenue']].tail()

In [ ]:
# Creating a score variable

df['score'] = (
    np.select(
        condlist=[(df['Revenue'].isna()) | (df['Revenue']<= 10000),
                  ((10000 < df['Revenue']) & (df['Revenue'] <= 50000)),
                  50000 < df['Revenue']],
        choicelist=["Low Revenue", "Medium Revenue", "High Revenue"],
        default=99999))


In [ ]:
df[['score', 'Revenue']].tail()